In [1]:
import numpy as np
import scipy.linalg as sla
import scipy.sparse as sps
import matplotlib.pyplot as plt

In [54]:
f = lambda x: 1.0 / np.sqrt(x)

N = 500

U, ss, _ = np.linalg.svd(np.random.randn(N,N))
A = U @ np.diag(ss) @ U.T
f_of_A = U @ np.diag(f(ss)) @ U.T

print('np.linalg.cond(A)=', np.linalg.cond(A))
print('cond(f_of_A)=', np.linalg.cond(f_of_A))

np.linalg.cond(A)= 9477.803166630298
cond(f_of_A)= 97.35400950460712


In [63]:
apply_A = lambda x: A @ x
b = np.random.randn(N)
x_true = f_of_A @ b

qq = [np.zeros(b.shape), b / np.linalg.norm(b)]
alphas = []
betas = [0.0]

for ii in range(50):
    Q = np.asarray(qq[1:]).T.reshape((N, -1))
    
    q_prev = qq[-2]
    q_cur = qq[-1]
    v = apply_A(q_cur)
    alpha = q_cur.T @ v
    v = v - betas[-1] * q_prev - alpha * q_cur
    beta = np.linalg.norm(v)
    q_next = v / beta
    qq.append(q_next)
    alphas.append(alpha)
    betas.append(beta)
    
    evals_T, evecs_T = sla.eigh_tridiagonal(alphas, betas[1:-1])
    f_of_T = evecs_T @ np.diag(f(evals_T)) @ evecs_T.T
    x = Q @ (f_of_T @ (Q.T @ b))
    relres = np.linalg.norm(x - f_of_A @ b) / np.linalg.norm(b)
    relerr = np.linalg.norm(x_true - x) / np.linalg.norm(x_true)
    print('ii=', ii, ', relres=', relres, ', relerr=', relerr)
    
T = sps.diags([betas[1:-1], alphas, betas[1:-1]], [-1, 0, 1])

ii= 0 , relres= 0.306556742750443 , relerr= 0.694569523795415
ii= 1 , relres= 0.2524666600039678 , relerr= 0.5720169331128551
ii= 2 , relres= 0.21423092075820335 , relerr= 0.48538573080550385
ii= 3 , relres= 0.1831691359734617 , relerr= 0.41500864866206266
ii= 4 , relres= 0.15582265749071744 , relerr= 0.3530493834153556
ii= 5 , relres= 0.130303952341913 , relerr= 0.29523132753423076
ii= 6 , relres= 0.10770507952164605 , relerr= 0.24402877301771211
ii= 7 , relres= 0.0899568430246103 , relerr= 0.2038163671141505
ii= 8 , relres= 0.07625555018065798 , relerr= 0.17277317308545959
ii= 9 , relres= 0.06510637280304214 , relerr= 0.1475123396345207
ii= 10 , relres= 0.05624877848970289 , relerr= 0.12744357517352395
ii= 11 , relres= 0.04803782997852984 , relerr= 0.1088399243578679
ii= 12 , relres= 0.041575245489971775 , relerr= 0.0941975642178415
ii= 13 , relres= 0.037237595654119014 , relerr= 0.084369695635182
ii= 14 , relres= 0.033366366471949065 , relerr= 0.07559860227922348
ii= 15 , relres= 0.